In [1]:
import numpy as np
import math
from GRAPE import *
import matplotlib.pyplot as plt
from qiskit import Aer, execute
import warnings
warnings.filterwarnings('ignore')

In [2]:
H = math.sqrt(0.5)*np.array([1,1,1,-1]).reshape(2,2)
X = np.array([0,1,1,0]).reshape(2,2)
Y = np.array([0,-1j,1j,0]).reshape(2,2)
Z = np.array([1,0,0,-1]).reshape(2,2)
U = np.linalg.eig(H)[1].conj().T

CX = np.eye(4)
CX[2][2] = 0
CX[3][3] = 0
CX[2][3] = 1
CX[3][2] = 1

CZ = np.eye(4)
CZ[3][3] = -1

CCZ = np.eye(8)
CCZ[7][7] = -1

TOFFOLI = np.eye(8)
TOFFOLI[6][6] = 0
TOFFOLI[7][7] = 0
TOFFOLI[6][7] = 1
TOFFOLI[7][6] = 1

U1 = np.kron(np.kron(X,U),np.eye(2))
V1 = CCZ
Ud1 = U1.conj().T

U2 = np.kron(np.kron(X,U),np.eye(2))
V2 = np.kron(CZ,np.eye(2))
Ud2 = U2.conj().T


U3 = np.kron(np.kron(np.eye(2),U),np.eye(2))
V3 = np.kron(np.eye(2),CZ)
Ud3 = U3.conj().T

U4 = np.kron(np.kron(np.eye(2),U),np.eye(2))
V4 = np.kron(CZ,np.eye(2)) @ np.kron(np.eye(2),CZ)
Ud4 = U4.conj().T

U5 = np.kron(np.kron(np.eye(2),X @ U),np.eye(2))
V5 = CCZ
Ud5 = U5.conj().T

J = np.zeros((3, 3))
J[0][1] = 0.1385
J[1][2] = 0.01304
J[0][2] = 0.00148

gates = [[U1, V1, Ud1], [U2, V2, Ud2], [U3, V3, Ud3], [U4, V4, Ud4], [U5, V5, Ud5]]

In [3]:
indices = ["U", "V", "Ud"]
for number, index in []:
    if True:
        print(number, index, ":\n")

        n = 200
        m = 200
        k = 6

        descents = [GradientDescent(gates[number - 1][index], k) for _ in range(n)]

        for descent in descents:
            descent.noise = 1
            descent.randomize_params()
            descent.set_j(J)

        losses = [descent.descend(steps=m, track_distance=True, time_sensitive=True) for descent in descents]

        #for loss in losses:
         #   plt.plot(np.real(loss[10:]))


        t_min = 600
        p_min = -1
        for i in range(n):
            if descents[i].time < 600 and descents[i].distance < 1:
                descents[i].make_times_positive()
                if descents[i].time < t_min:
                    t_min = descents[i].time
                    p_min = i
                #print("i: ", i, " ; distance:", descents[i].distance.round(6).real, "; time:", descents[i].time.round(2))
        print(f"Choose descent {p_min} : time={descents[p_min].time}, distance={descents[p_min].distance}")


        descent = descents[p_min]

        loss = descent.descend(steps=4000, track_distance=True, time_sensitive=False)
        plt.plot(np.real(loss))

        print(f"Distance: {descent.distance.real}; Time: {descent.time.real}")
        descent.to_text(f"./Oleg/{indices[index]}{number}.oleg")





In [5]:
indices = ["U", "V", "Ud"]
for number in [1, 2, 3, 4, 5]:
    for index in [0, 1, 2]:
        descent = GradientDescent(gates[number - 1][index], filename=f"./Oleg/{indices[index]}{number}.oleg")
        print(f"Was :{descent.distance}")
        while descent.distance > 0.01:
            descent.descend(steps=200, track_distance=False, time_sensitive=False)
        print(f"Now :{descent.distance}")
        descent.to_text(f"./Oleg/{indices[index]}{number}.device")

Was :(0.009982827294648167+0j)
Now :(0.009982827294648167+0j)
Was :(0.00947231898449203+0j)
Now :(0.00947231898449203+0j)
Was :(0.009536724343726835+0j)
Now :(0.009536724343726835+0j)
Was :(0.009998571881879834+0j)
Now :(0.009998571881879834+0j)
Was :(0.009952810155975881+0j)
Now :(0.009952810155975881+0j)
Was :(0.00958615927608204+0j)
Now :(0.00958615927608204+0j)
Was :(0.009961730279575759+0j)
Now :(0.009961730279575759+0j)
Was :(0.00993125123128913+0j)
Now :(0.00993125123128913+0j)
Was :(0.009302207506194309+0j)
Now :(0.009302207506194309+0j)
Was :(0.009446523739724946+0j)
Now :(0.009446523739724946+0j)
Was :(0.009991791409239614+0j)
Now :(0.009991791409239614+0j)
Was :(0.006584869707886002+0j)
Now :(0.006584869707886002+0j)
Was :(0.0046353111930881215+0j)
Now :(0.0046353111930881215+0j)
Was :(0.009999293439625807+0j)
Now :(0.009999293439625807+0j)
Was :(0.009981562141335658+0j)
Now :(0.009981562141335658+0j)
